In [ ]:
from bomberman.Environment import Environnement
from tqdm import trange
import torch
from DQN_Agent import BuffedDQNAgent

class Trainer():
	def __init__(self, Agent : BuffedDQNAgent) -> None:
		self.training_agent = Agent(1)
		self.env_1 = Environnement(1)

		self.to_beat_agent = Agent(2)
		self.env_2 = Environnement(2)

	def reset(self):
		# Player 1
		self.env_1.reset()
		state_1 = self.env_1.get_state()

		# Player 2
		self.env_2.reset()
		state_2 = self.env_2.get_state()
		return state_1, state_2
		
	def compute_reward(self, old_state, next_state):
		# TODO
		return 0

	def train(self, steps=10, log_each=1, skip_frames=4, reset_agent_each=10):
		state_1, state_2 = self.reset()
		current_skip = skip_frames
		
		i = 1

		while i <= steps:
			game_over = False

			data_point = {"obs": state_2} # need [obs, action, reward, next_obs, done]

			while current_skip > 0 and not game_over:
				with torch.no_grad(): # Don't compute grad for game playing
					# To_beat_agent turn
					tobeat_action = self.to_beat_agent.get_action(state_1)
					state_1 = self.env_1.do_action(tobeat_action)
					# Then the training agent turn
					training_action = self.training_agent.get_action(state_2)
					state_2 = self.env_2.do_action(training_action)

					# if it is the first step of the observation
					if current_skip == skip_frames:
						data_point["action"] = training_action 

					current_skip -= 1

					if (state_2.winner is not None):
						game_over = True
			
			data_point["next_obs"] = state_2
			data_point["reward"] = self.compute_reward(data_point["obs"], data_point["next_obs"])
			data_point["done"] = game_over

			if game_over:
				state_1, state_2 = self.reset()

			# TODO : add to replay buffer and train with replay buffer

			if i % reset_agent_each == 0: # reset the to beat agent to the current trained agent
				self.to_beat_agent.brain.load_state_dict(self.training_agent.brain.state_dict())

			i += 1 # TODO : only update i when a step of training is done with the memory
			print(f"{steps} steps left")